In [2]:
from photerr import LsstErrorModel
import pandas as pd
import numpy as np

In [3]:
noiseless_photometry = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_data.npy")[:1000000, :6]
sps_params = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_params.npy")[:1000000, :]
catalog = pd.DataFrame(noiseless_photometry, columns=['u', 'g', 'r', 'i', 'z', 'y'])

In [4]:
random_state=42
errModel = LsstErrorModel(sigLim=0, absFlux=True)
sig5detections = LsstErrorModel(sigLim=5)
sig2detections = LsstErrorModel(sigLim=2)

observed_catalog = errModel(catalog, random_state=random_state).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)

catalog_sig5 = sig5detections(observed_catalog, random_state=random_state).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)
catalog_sig5.rename(columns={"u": "u5", "g": "g5", "r": "r5", "i": "i5", "z": "z5", "y": "y5" }, inplace=True)

catalog_sig2 = sig2detections(observed_catalog, random_state=random_state).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)
catalog_sig2.rename(columns={"u": "u2", "g": "g2", "r": "r2", "i": "i2", "z": "z2", "y": "y2" }, inplace=True)

observed_catalog = observed_catalog.join(catalog_sig5).join(catalog_sig2)

In [5]:
observed_catalog

,u,g,r,i,z,y,u5,g5,r5,i5,z5,y5,u2,g2,r2,i2,z2,y2
0,29.077212,29.006471,29.355314,28.511281,27.365579,24.419026,NaN,NaN,NaN,NaN,NaN,24.552852,NaN,NaN,NaN,27.841580,NaN,24.552852
1,20.661355,20.635699,20.163962,20.311902,20.371887,20.442743,20.660658,20.637301,20.164046,20.316226,20.367368,20.438331,20.660658,20.637301,20.164046,20.316226,20.367368,20.438331
2,27.425175,26.793837,26.826197,27.103724,26.704071,28.643803,NaN,26.649376,26.763150,NaN,NaN,NaN,NaN,26.649376,26.763150,27.364204,26.586722,NaN
3,27.929121,32.361441,30.941260,29.078675,27.727271,29.364477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.990785,NaN
4,18.313505,18.286843,17.997658,18.049663,18.083339,18.031653,18.315667,18.288610,17.994992,18.047832,18.081270,18.029488,18.315667,18.288610,17.994992,18.047832,18.081270,18.029488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,28.282427,30.651675,27.325318,24.460094,23.437816,22.793269,NaN,NaN,NaN,24.484156,23.442475,22.801848,NaN,NaN,27.496985,24.484156,23.442475,22.801848
999996,18.015375,17.789425,17.888425,17.395390,17.902992,18.127446,18.014205,17.785061,17.884376,17.399229,17.900952,18.134566,18.014205,17.785061,17.884376,17.399229,17.900952,18.134566
999997,28.420445,23.677204,22.562164,21.599041,20.988389,20.538090,NaN,23.685637,22.564289,21.604504,20.993445,20.538770,NaN,23.685637,22.564289,21.604504,20.993445,20.538770
999998,29.333022,33.150241,30.614674,28.065118,26.507004,25.681328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.568077,25.380253


In [6]:
brightness_cut = 22
observed_catalog.dropna(axis=0, subset=['i5'], inplace=True) #require 5sigma detection in i band
observed_catalog.drop(observed_catalog[observed_catalog['i5'] < brightness_cut].index, inplace=True)
observed_catalog


,u,g,r,i,z,y,u5,g5,r5,i5,z5,y5,u2,g2,r2,i2,z2,y2
9,28.607734,29.551780,26.678613,25.269228,24.819414,24.586305,NaN,NaN,26.712498,25.196724,24.872299,24.482306,NaN,NaN,26.712498,25.196724,24.872299,24.482306
15,27.711542,27.140973,25.679089,24.503926,23.591683,25.233829,NaN,27.159701,25.699873,24.505942,23.625896,NaN,NaN,27.159701,25.699873,24.505942,23.625896,25.579006
16,26.006904,24.272420,23.578712,23.257801,23.050582,22.889766,NaN,24.287070,23.575223,23.266378,23.055448,22.858651,26.329785,24.287070,23.575223,23.266378,23.055448,22.858651
20,27.763630,30.878937,28.025620,25.476961,25.195060,25.093675,NaN,NaN,NaN,25.397578,25.127067,25.031392,NaN,NaN,27.937258,25.397578,25.127067,25.031392
22,28.281610,31.848635,25.829620,22.948710,22.547465,22.218772,NaN,NaN,25.914900,22.956728,22.544636,22.207224,NaN,NaN,25.914900,22.956728,22.544636,22.207224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999984,26.834699,26.243390,25.657850,25.153262,25.333205,23.024671,NaN,26.206933,25.697172,25.112653,25.507161,23.027836,26.653352,26.206933,25.697172,25.112653,25.507161,23.027836
999991,28.714593,28.653106,27.151838,26.205105,25.286644,24.689674,NaN,NaN,NaN,26.453254,25.292418,24.825582,NaN,NaN,27.313013,26.453254,25.292418,24.825582
999993,28.173934,27.287871,26.684982,25.835360,24.417397,24.026091,NaN,26.956333,26.635909,25.726216,24.395832,23.985290,NaN,26.956333,26.635909,25.726216,24.395832,23.985290
999995,28.282427,30.651675,27.325318,24.460094,23.437816,22.793269,NaN,NaN,NaN,24.484156,23.442475,22.801848,NaN,NaN,27.496985,24.484156,23.442475,22.801848


In [7]:
def select_u_dropouts(observed_catalog):
    
    udrop = observed_catalog.copy(deep=True)
    udrop = udrop.dropna(axis=0, subset=['i5'])
    #udrop = udrop.dropna(axis=0, subset=['r5'])
    #udrop = udrop.dropna(axis=0, subset=['g5'])
    #udrop = udrop.drop(udrop[np.isnan(udrop.u2) == False].index)

    return udrop#.filter(['u5','g5','r5','i5','z5','y5'])

def select_g_dropouts(observed_catalog):
    
    gdrop = observed_catalog.copy(deep=True)
    gdrop = gdrop.dropna(axis=0, subset=['i5'])
    #gdrop = gdrop.dropna(axis=0, subset=['r5'])
    #gdrop = gdrop.drop(gdrop[np.isnan(gdrop.g2) == False].index)
    gdrop = gdrop.drop(gdrop[np.isnan(gdrop.u2) == False].index)

    
    return gdrop.filter(['u5','g5','r5','i5','z5','y5'])

def select_r_dropouts(observed_catalog):

    rdrop = observed_catalog.copy(deep=True)
    #rdrop = rdrop.dropna(axis=0, subset=['z5'])
    rdrop = rdrop.dropna(axis=0, subset=['i5'])
    #rdrop = rdrop.drop(rdrop[np.isnan(rdrop.r2) == False].index)
    rdrop = rdrop.drop(rdrop[np.isnan(rdrop.g2) == False].index)
    
    return rdrop.filter(['u5','g5','r5','i5','z5','y5'])

In [8]:
select_u_dropouts(observed_catalog)

,u,g,r,i,z,y,u5,g5,r5,i5,z5,y5,u2,g2,r2,i2,z2,y2
9,28.607734,29.551780,26.678613,25.269228,24.819414,24.586305,NaN,NaN,26.712498,25.196724,24.872299,24.482306,NaN,NaN,26.712498,25.196724,24.872299,24.482306
15,27.711542,27.140973,25.679089,24.503926,23.591683,25.233829,NaN,27.159701,25.699873,24.505942,23.625896,NaN,NaN,27.159701,25.699873,24.505942,23.625896,25.579006
16,26.006904,24.272420,23.578712,23.257801,23.050582,22.889766,NaN,24.287070,23.575223,23.266378,23.055448,22.858651,26.329785,24.287070,23.575223,23.266378,23.055448,22.858651
20,27.763630,30.878937,28.025620,25.476961,25.195060,25.093675,NaN,NaN,NaN,25.397578,25.127067,25.031392,NaN,NaN,27.937258,25.397578,25.127067,25.031392
22,28.281610,31.848635,25.829620,22.948710,22.547465,22.218772,NaN,NaN,25.914900,22.956728,22.544636,22.207224,NaN,NaN,25.914900,22.956728,22.544636,22.207224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999984,26.834699,26.243390,25.657850,25.153262,25.333205,23.024671,NaN,26.206933,25.697172,25.112653,25.507161,23.027836,26.653352,26.206933,25.697172,25.112653,25.507161,23.027836
999991,28.714593,28.653106,27.151838,26.205105,25.286644,24.689674,NaN,NaN,NaN,26.453254,25.292418,24.825582,NaN,NaN,27.313013,26.453254,25.292418,24.825582
999993,28.173934,27.287871,26.684982,25.835360,24.417397,24.026091,NaN,26.956333,26.635909,25.726216,24.395832,23.985290,NaN,26.956333,26.635909,25.726216,24.395832,23.985290
999995,28.282427,30.651675,27.325318,24.460094,23.437816,22.793269,NaN,NaN,NaN,24.484156,23.442475,22.801848,NaN,NaN,27.496985,24.484156,23.442475,22.801848


In [10]:
np.nan - 23

nan